In [39]:
input_file_path = 'SMI_Historical_Data(10Nov).csv'
output_file_path = 'SMI_log_returns(10Nov).csv'

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

# Function to process CSV files
def process_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        # Perform some processing on the DataFrame
        df.fillna(0, inplace=True)  # Example: Fill NaN values with 0
        return df
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Example usage
processed_df = process_csv(input_file_path)

# Remove ,'s and convert to numeric
def to_numeric_remove_commas(series):
    return pd.to_numeric(series.str.replace(',', ''))

processed_df['Date'] = pd.to_datetime(processed_df['Date'])

# Converting each column data type
if 'Price' in processed_df.columns:
    processed_df['Close'] = to_numeric_remove_commas(processed_df['Price'])
    del processed_df['Price']
else:
    processed_df['Close'] = to_numeric_remove_commas(processed_df['Close'])

processed_df['Open'] = to_numeric_remove_commas(processed_df['Open'])
processed_df['High'] = to_numeric_remove_commas(processed_df['High'])
processed_df['Low'] = to_numeric_remove_commas(processed_df['Low'])

print(processed_df.head())

        Date      Open      High       Low    Vol. Change %     Close
0 2024-11-08  11918.78  11932.08  11775.75  24.89M   -1.00%  11797.72
1 2024-11-07  11863.67  11990.69  11855.47  29.30M    0.59%  11917.00
2 2024-11-06  11997.42  12122.20  11821.48  37.28M   -0.16%  11847.29
3 2024-11-05  11920.49  11924.70  11839.35  23.35M   -0.26%  11866.01
4 2024-11-04  11946.40  11983.26  11896.94  16.63M   -0.59%  11896.94


In [41]:
new_df = processed_df[['Date', 'Close']].copy()
closing_prices = new_df['Close']

log_returns = np.log(closing_prices / closing_prices.shift(1))
new_df['log_returns'] = log_returns
new_df['log_prices'] = np.log(closing_prices)

print("new_df")
print(new_df.head())
print()

print("closing_prices")
print(closing_prices.head())

new_df
        Date     Close  log_returns  log_prices
0 2024-11-08  11797.72          NaN    9.375662
1 2024-11-07  11917.00     0.010060    9.385721
2 2024-11-06  11847.29    -0.005867    9.379854
3 2024-11-05  11866.01     0.001579    9.381433
4 2024-11-04  11896.94     0.002603    9.384037

closing_prices
0    11797.72
1    11917.00
2    11847.29
3    11866.01
4    11896.94
Name: Close, dtype: float64


In [ ]:
# Get new dataframe into a csv file
new_df.to_csv(output_file_path, index=False)